# Scratch pad StatsModels

The purpose of this notebook is to explore single-factor and multiple-factor regression using StatsModels.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.axes as axes
import statsmodels.api as sm
%matplotlib inline

In [ ]:
c = cm.Paired.colors
# c[0] c[1] ... c[11]
# See "paired" in "qualitative colormaps"
# https://matplotlib.org/tutorials/colors/colormaps.html

In [ ]:
# Function to remove upper and right graphic borders.
def despine(ax: axes.Axes) -> None:
    'Remove the top and right spines of a graph'
    for spine in 'right', 'top':
        ax.spines[spine].set_color('none')

## Example from stock market prices

In [ ]:
# Create the dataframe.
# The dependent variable is Stock_Index_Price.
# The independent variables are Interest_Rate and
# Unemployment_Rate.
stock_market = {'Year': [2017,2017,2017,2017,2017,2017,2017,
                         2017,2017,2017,2017,2017,2016,2016,
                         2016,2016,2016,2016,2016,2016,2016,
                         2016,2016,2016],
                'Month': [12, 11,10,9,8,7,6,5,4,3,2,1,12,11,
                          10,9,8,7,6,5,4,3,2,1],
                'Interest_Rate': [2.75,2.5,2.5,2.5,2.5,2.5,
                                  2.5,2.25,2.25,2.25,2,2,2,
                                  1.75,1.75,1.75,1.75,1.75,
                                  1.75,1.75,1.75,1.75,1.75,1.75],
                'Unemployment_Rate': [5.3,5.3,5.3,5.3,5.4,5.6,
                                      5.5,5.5,5.5,5.6,5.7,5.9,
                                      6,5.9,5.8,6.1,6.2,6.1,
                                      6.1,6.1,5.9,6.2,6.2,6.1],
                'Stock_Index_Price': [1464,1394,1357,1293,1256,
                                      1254,1234,1195,1159,1167,
                                      1130,1075,1047,965,943,
                                      958,971,949,884,866,876,
                                      822,704,719]        
                }
df = pd.DataFrame(stock_market,columns=['Year','Month',
                                        'Interest_Rate',
                                        'Unemployment_Rate',
                                        'Stock_Index_Price'])

In [ ]:
# Scatter plot of Y v. X.
for xx, yy in ('Interest_Rate', 'Stock_Index_Price'),\
              ('Interest_Rate', 'Unemployment_Rate'):
    ax = df.plot.line(x=xx,
                      y=yy,
                      legend=False,
                      style='.',
                      color=c[1],
                      figsize=(9, 6))
    ax.autoscale(enable=True)
    ax.set_title(f'{yy} vs {xx}', fontweight='bold')
    ax.set_ylabel(f'{yy}', fontweight='bold')
    ax.set_xlabel(f'{xx}', fontweight='bold')
    despine(ax)

In [ ]:
x = df[['Interest_Rate', 'Unemployment_Rate']]
y = df['Stock_Index_Price']
x = sm.add_constant(x)
model = sm.OLS(y, x, missing='drop').fit()
predictions = model.predict(x)
model.summary2()

In [ ]:
df['Predicted_Stock_Index_Price'] = predictions

In [ ]:
model.params.round(3)

In [ ]:
model.pvalues.round(3)

In [ ]:
model.rsquared_adj.round(3)

In [ ]:
model.conf_int().round(3)

In [ ]:
np.sqrt(model.mse_resid).round(3)

In [ ]:
np.sqrt(model.mse_model).round(3)

In [ ]:
np.sqrt(model.mse_total).round(3)

## Example from Boston house prices

In [ ]:
features_cols = pd.read_csv('boston_features.csv', nrows=1).columns
features = pd.read_csv('boston_features.csv', usecols=features_cols[1:])
features.head()

In [ ]:
target_cols = pd.read_csv('boston_target.csv', nrows=1).columns
target = pd.read_csv('boston_target.csv', usecols=target_cols[1:])
target.head()

In [ ]:
boston = pd.concat([target, features], axis=1, sort=False)
boston.head()

In [ ]:
# Scatter plot of Y v. X.
for xx, yy in ('RM', 'MEDV'),\
              ('LSTAT', 'MEDV'):
    ax = boston.plot.line(x=xx,
                      y=yy,
                      legend=False,
                      style='.',
                      color=c[1],
                      figsize=(9, 6))
    ax.autoscale(enable=True)
    ax.set_title(f'{yy} vs {xx}', fontweight='bold')
    ax.set_ylabel(f'{yy}', fontweight='bold')
    ax.set_xlabel(f'{xx}', fontweight='bold')
    despine(ax)

In [ ]:
x = features[['RM', 'LSTAT']]
y = target['MEDV']
x = sm.add_constant(x)

In [ ]:
model = sm.OLS(y, x, missing='drop').fit()
predictions = model.predict(x)
model.summary2()

In [ ]:
df['Predicted_MEDV'] = predictions

In [ ]:
model.params.round(3)

In [ ]:
model.pvalues.round(3)

In [ ]:
model.rsquared_adj.round(3)

In [ ]:
model.conf_int().round(3)

In [ ]:
np.sqrt(model.mse_resid).round(3)

In [ ]:
np.sqrt(model.mse_model).round(3)

In [ ]:
np.sqrt(model.mse_total).round(3)